### CARGAR DEPENDENCIAS

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from model import *
import os
import torchvision.transforms.functional as F
from inputs.xbox_controller_emulator import XboxControllerEmulator
from UDP_listener import udp_listener
from ScreenRecorder import *
from torchvision import transforms, models
import time
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import random

### DEFINICION DE HIPERPÁRAMETROS

In [10]:
# Parámetros de captura de pantalla
screen_size = (1920, 1080)
full_screen = True
fps = 30 # HAY QUE MEDIR LA CAPACIDAD Y AJUSTAR ESTE VALOR

# Hiperparametros del modelo
name = "CNNminitest"
architecture = "CNN_RNN"    # "CNN", "CNN_RNN", "CNN_LSTM_STATE"
output_size = 2             # Giro y aceleración

# Hiperparametros de la CNN
cnn_name = "efficientnet_b0"#"efficientnet_v2_s", "efficientnet_b0", "efficientnet_b1", "efficientnet_b2", "efficientnet_b3"
input_size = (240, 135)     # 16:9 ratio
dropout = 0 # 0.5
bias = True                 # Si se desea usar bias en las capas convolucionales
cnn_train = True            # Si se desea entrenar la parte CNN

# Hiperparametros de la RNN/LSTM
hidden_size = 256           # Número de neuronas en la capa oculta de la RNN o LSTM 512 usado por Iker
num_layers = 1              # Número de capas en la RNN o LSTM
seq_len = 1                 # Número de imágenes a considerar en la secuencia

# Hiperparametros de SAC
learning_rate = 3e-4   # Tasa de aprendizaje para el optimizador
discount_factor = 0.99 # Factor de descuento para las recompensas futuras
alpha = 0.2            # Parámetro de entropía para SAC (controla la exploración)
tau = 0.005            # Parámetro de actualización suave de las redes objetivo
batch_size = 64        # Tamaño de batch para actualizar el agente

# Parámetros de recompensas
reward_speed_weight = 1.0        # Peso de la velocidad en la recompensa
reward_track_position_weight = 1.0  # Peso de la posición en la pista
reward_laps_weight = 100.0         # Peso de las vueltas completadas DEBE SER ALTO
penalty_tyres_out = -1.0        # Penalización por salirse de la pista
penalty_car_damage = -2.0       # Penalización por dañar el auto

# Otras configuraciones
max_steps_per_episode = 1000   # Máximo número de pasos por episodio
num_episodes = 500             # Número de episodios de entrenamiento

save_dir = f"./trained_models/{name}"

# Definir el nombre del modelo a guardar
model_name = f"{name}-{architecture}-{cnn_name}-{seq_len}-{input_size[0]}-{input_size[1]}-{output_size}-{hidden_size}"
print(f"{model_name}")

os.makedirs(save_dir, exist_ok=True) # Crear directorio de guardado si no existe

# Definir el escritor de TensorBoard para visualización
writer = SummaryWriter(log_dir="./runs/" + model_name) 

CNNminitest-CNN_RNN-efficientnet_b0-5-240-135-2-256-epoch


### FUNCIONES AUXILIARES

In [ ]:
# Función para calcular la recompensa en cada paso
def calculate_reward(variables):
    """Calcula la recompensa basándose en las variables del entorno"""

    speed = variables["speed"]
    track_position = variables["track_position"]
    laps = variables["laps"]
    tyres_out = variables["tyres_out"]
    car_damage = variables["car_damage"]
    
    # Recompensa por velocidad y posición en la pista
    reward = reward_speed_weight * speed + reward_track_position_weight * track_position + reward_laps_weight * laps
    
    # PULIR ESTA PARTE, HAY QUE HACER QUE LA RECOMPENSA POR VELOCIDAD SOLO SEA PARA QUE LA VELOCIDAD NO SEA 0, 
    # NO RECOMPENSAR POR VELOCIDAD ALTA YA QUE PUEDE PERDER EL CONTROL
    # PONER TIMER A LAS RUEDAS FUERA DE LA PISTA, PENALIZAR POCO SI SOLO ES 1 O 2 PERO PENALIZAR MUCHO SI ES MAS, 
    # TAL VEZ HACERLO DE FORMA EXPONENCIAL AL TIEMPO Y A LA CANTIDAD DE RUEDAS FUERA

    # Penalización por salirse de la pista y por daño
    if tyres_out > 0:
        reward += penalty_tyres_out
    if car_damage > 0:
        reward += penalty_car_damage

    return reward

# Función para reiniciar el entorno (por ejemplo, cuando el auto se sale de la pista)
def reset_environment():
    """Reinicia la simulación (puedes personalizar este método según el juego)"""
    # Lógica para reiniciar el entorno, como reiniciar la carrera o posición

    # PONER UN TIMER PARA QUE NO CONTINUE INMEDIATAMENTE HASTA QUE EL AUTO ESTE EN UNA POSICION LISTA PARA INICIAR
    pass

# Definir las transformaciones
transform = transforms.Compose([
    transforms.Resize((input_size)),  # Cambia el tamaño de las imágenes a (height x width)
    transforms.ToTensor(),         # Convierte las imágenes a tensores
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizar la imagen
])

### INICIAR MODELO Y CARGAR DATOS

In [11]:
# Inicializar el modelo

if architecture == "CNN":
    model = CNN(cnn_name, output_size, (3, *input_size), dropout, bias, cnn_train)
    seq_len = 1
elif architecture == "CNN_RNN":
    model = CNN_RNN(cnn_name, hidden_size, output_size, (3, *input_size), num_layers, dropout, bias)
elif architecture == "CNN_LSTM_STATE":
    model = CNN_LSTM_STATE(cnn_name, hidden_size, output_size, (3, *input_size), num_layers, dropout, bias)
    hidden_state = model.init_hidden(batch_size)

if full_screen:
    region = {'left': 0, 'top': 0, 'width': screen_size[0], 'height': screen_size[1]}
else:
    region = {'left': 0, 'top': 40, 'width': screen_size[0], 'height': screen_size[1]}

# Cargar el modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Usando CUDA" if torch.cuda.is_available() else "USANDO CPU")


# Optimización y función de pérdida
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

criterion = nn.MSELoss()

Usando CUDA


### ENTRENAMIENTO

In [12]:
print("Iniciando entrenamiento...")

finished = False
episodio = 0
sequence_buffer = []  # Buffer para almacenar las imágenes de la secuencia en caso de usar RNN o LSTM

start_time = time.time()    # Tiempo de inicio del entrenamiento

# Iniciar el entrenamiento

try:
    while not finished: #Ciclos de episodios
        print(f"Episodio {episodio}", end="\r")
        episodio += 1

        # Reiniciar el entorno
        reset_environment()
        episode_start_time = time.time()
        total_reward = 0
        running = True
        model.train()
        
        while running: #Ciclos de pasos
            step_start_time = time.time()
            # Capturar la pantalla   
            img = capture_screen(region)           
            preprocessed_img = Image.fromarray(img.astype(np.uint8)).convert('RGB')# Convertir la imagen preprocesada a un objeto PIL y aplicar las transformaciones
            preprocessed_img = transform(preprocessed_img)

            # Telemetria del juego
            variables = udp_listener()

            # Añadir la imagen preprocesada al buffer de secuencia
            sequence_buffer.append(preprocessed_img)
        
            # Mantener solo las últimas imágenes en el buffer
            if len(sequence_buffer) > seq_len:
                sequence_buffer.pop(0)

            # Verificar si tenemos suficientes imágenes para una secuencia completa
            if len(sequence_buffer) == seq_len:
                # Convertir la secuencia de imágenes en un tensor
                sequence_tensor = torch.stack(sequence_buffer).unsqueeze(0).to(device)

                # Habilitar precisión mixta y para usar la GPU para entrenar
                with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                    if architecture == "CNN":
                        output = model(sequence_tensor)
                    elif architecture == "CNN_RNN":
                        outputs = model(sequence_tensor)
                    else:
                        print("Arquitectura no soportada")
                        #parar la ejecucion de todo el programa
                        raise SystemExit

                    # Calcular la recompensa en base a las variables
                    reward = calculate_reward(variables)

                    # 5. Aquí irían las decisiones del agente (acciones) usando SAC, por ejemplo:
                    # - Elegir acción basada en las características extraídas por la CNN
                    # - Actualizar las redes del actor y crítico según las fórmulas de SAC
                    # - Enviar comandos al juego, como dirección y aceleración

                    # 6. Aplicar retropropagación para optimizar la red
                    loss = ...  # Aquí debes definir cómo calculas la pérdida para SAC
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_reward += reward

            # 7. Condición para reiniciar el episodio (por ejemplo, si el auto se sale de la pista)
            if variables["tyres_out"] == 4 or variables["car_damage"] > 0:
                print("El auto se ha salido de la pista o ha sufrido daño, reiniciando episodio...")
                break  # Termina el episodio si el auto está fuera de la pista o dañado

            # EN ESTA PARTE PONER UN SLEEP PARA HACER QUE LOS STEPS ESTEN ESPACIADOS DE FORMA CONSTANTE
            # MEDIR EL TIEMPO QUE PUEDE DEMORAR CADA STEP


except KeyboardInterrupt:
    print("\nEntrenamiento interrumpido.")

print("Entrenamiento terminado.")

writer.close()  # Cerrar el escritor de TensorBoard

torch.cuda.empty_cache()
torch.cuda.ipc_collect()

Iniciando entrenamiento...
Trabajando en Epoch 1. Progreso: 100.00%
Validando modelo...
Validando en Epoch 1. Progreso: 100.00%
Epoch [1/30], Train Loss: 0.1852, Val Loss: 0.1454, Epoch Time: 00:01:04, Total Time: 00:01:04
Trabajando en Epoch 2. Progreso: 100.00%
Validando modelo...
Validando en Epoch 2. Progreso: 100.00%
Epoch [2/30], Train Loss: 0.0749, Val Loss: 0.1432, Epoch Time: 00:01:05, Total Time: 00:02:09
Trabajando en Epoch 3. Progreso: 100.00%
Validando modelo...
Validando en Epoch 3. Progreso: 100.00%
Epoch [3/30], Train Loss: 0.0592, Val Loss: 0.1219, Epoch Time: 00:01:05, Total Time: 00:03:14
Trabajando en Epoch 4. Progreso: 6.45%
Entrenamiento interrumpido.
Entrenamiento terminado.
